# Libraries

In [4]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mutual_info_score
import shutil
import random

# Importing data

In [2]:
data_dir = "/NAS/dumbo/protocoles/HAMSI/OurData4Mispel_T1Gado"

# Functions

In [ ]:
def plot_combined_histogram_mask(image1_path, image2_path, mask_path, bins=50):
    
    image1 = nib.load(image1_path).get_fdata()
    image2 = nib.load(image2_path).get_fdata()
    mask = nib.load(mask_path).get_fdata()

    filtered_values1 = image1[mask>0]
    filtered_values2 = image2[mask>0]

    min_value = min(np.min(filtered_values1), np.min(filtered_values2))
    max_value = max(np.max(filtered_values1), np.max(filtered_values2))

    plt.hist(filtered_values1.ravel(), bins=bins, range=(min_value, max_value), color='blue', alpha=0.5, label='Image 1')

    plt.hist(filtered_values2.ravel(), bins=bins, range=(min_value, max_value), color='red', alpha=0.5, label='Image 2')

    #plt.title('Combined Histogram of Image 1 and Image 2 (Values >= 0)')
    plt.title('Combined Histogram of Image 1 and Image 2')
    plt.grid(True)
    
    plt.legend()

    plt.show()

In [ ]:
def bland_altman_plot_final(data1, data2, maskfile):
    
    downsample_factor=25
    
    image1 = nib.load(data1).get_fdata()[::downsample_factor,::downsample_factor,::downsample_factor]
    image2 = nib.load(data2).get_fdata()[::downsample_factor,::downsample_factor,::downsample_factor]
    mask = nib.load(maskfile).get_fdata()[::downsample_factor,::downsample_factor,::downsample_factor]
    
    mean = (image1 + image2) / 2
    diff = image1 - image2

    mean_diff = np.mean(diff[mask>0])
    std_diff = np.std(diff[mask>0], ddof=1)
    lower_limit = mean_diff - 1.96 * std_diff
    upper_limit = mean_diff + 1.96 * std_diff

    plt.figure(figsize=(8, 6))
    plt.scatter(mean[mask>0], diff[mask>0], c='b', alpha=0.5, s=30)
    plt.axhline(mean_diff, color='red', linestyle='--', linewidth=2)
    plt.axhline(lower_limit, color='green', linestyle='--', linewidth=1)
    plt.axhline(upper_limit, color='green', linestyle='--', linewidth=1)
    plt.xlabel('Moyenne des deux mesures')
    plt.ylabel('Différence entre les mesures')
    plt.title('Bland-Altman Plot')
    plt.grid(True)

    plt.show()
    
    points_within_limits = np.sum((diff[mask>0] >= lower_limit) & (diff[mask>0] <= upper_limit))
    all_points = np.sum(mask>0)

    return print(f"Number of points within Bland-Altman limits: {points_within_limits} / {all_points}")

# Slicing data

# Normalizing with z-score

# Mesure de similarité

### Bland atman plot

### Correlation coefficent

### Mutual information

# Diffusion model